In [6]:
import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from ioh_xplainer import explainer
import pandas as pd

from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
from IPython.display import display


df1 = pd.read_csv("intermediate-cma-1.csv")
df2 = pd.read_csv("intermediate-cma-2.csv")
df3 = pd.read_csv("intermediate-cma-3.csv")

dfall = pd.concat([ df1, df2, df3], ignore_index=True)

dfall = dfall.drop(columns=['Unnamed: 0'])  


#remove all mu > lambda

dfall = dfall[dfall['mu'] <= dfall['lambda_']]
dfall.loc[(dfall['lambda_'] == 'nan') & (dfall['dim'] == 5),'lambda_'] = 6
dfall.loc[(dfall['lambda_'] == 'nan') & (dfall['dim'] == 30),'lambda_'] = 8

dfall.loc[dfall['mu'] == np.nan,'mu'] = dfall.loc[dfall['mu'] == 'nan','lambda_'] // 2
print(dfall['mu'])
dfall.describe()
# #replacing stuff to fix
# dfall['mutation_reference'] = dfall['mutation_reference'].replace(np.nan,'nan')
# dfall['adaptation_method'] = dfall['adaptation_method'].replace(np.nan,'nan')

# dfall['lambda_'] = dfall['lambda_'].replace('2', 2.0)
# dfall['lambda_'] = dfall['lambda_'].replace('10', 10.0)

# dfall.loc[(dfall['lambda_'] == 10.0) & (dfall['dim'] == 30),'lambda_'] = 300
# dfall.loc[(dfall['lambda_'] == 10.0) & (dfall['dim'] == 5),'lambda_'] = 150
# dfall.loc[(dfall['lambda_'] == 2.0) & (dfall['dim'] == 30),'lambda_'] = 60
# dfall.loc[(dfall['lambda_'] == 2.0) & (dfall['dim'] == 5),'lambda_'] = 10
# dfall.loc[(dfall['lambda_'] == 'nan') & (dfall['dim'] == 5),'lambda_'] = 6
# dfall.loc[(dfall['lambda_'] == 'nan') & (dfall['dim'] == 30),'lambda_'] = 8

dfall.to_pickle("cma_final.pkl")
#print(dfall['lambda_'].describe())

72000       5.0
72001       5.0
72002       5.0
72003       5.0
72004       5.0
           ... 
7464955    20.0
7464956    20.0
7464957    20.0
7464958    20.0
7464959    20.0
Name: mu, Length: 2799360, dtype: float64


In [8]:
import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from ioh_xplainer import explainer
import pandas as pd

from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
from IPython.display import display



data_file = "cma_final.pkl"
features = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation', 'lambda_', 'mu']
df = pd.read_pickle(data_file)


config_dict = {}
for f in features:
    config_dict[f] = list(map(str, df[f].unique()))

config_dict['elitist'] = [False, True]
config_dict['active'] = [False, True]

print(config_dict)
print( df['dim'].unique())
print( df['seed'].unique())
print( df['iid'].unique())

cs = ConfigurationSpace(config_dict)

print(cs)
print( df['dim'].unique())
cmaes_explainer = explainer(None, 
                 cs , 
                 algname="mod-CMA",
                 dims = [5,30],#, 10, 20, 40 
                 fids = np.arange(1,25), #,5
                 iids = df['iid'].unique(), #20 
                 reps = len( df['seed'].unique()), 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = {},
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)


cmaes_explainer.load_results(data_file)
#for feature in features:
#    cmaes_explainer.df[feature] = cmaes_explainer.df[feature].astype("category")
df = cmaes_explainer.performance_stats()
display(df.style.bar(cmap='viridis'))

#cmaes_explainer.plot(partial_dependence=False, best_config=False)
#print(de_explainer.stats[5])
#df = de_explainer.stats[5]
#print(df.to_latex(index=False,
#                  float_format="{:.2f}".format,
#                  multicolumn_format = "c"
#))  


{'elitist': [False, True], 'mirrored': ['nan', 'mirrored', 'mirrored pairwise'], 'base_sampler': ['sobol', 'gaussian', 'halton'], 'weights_option': ['default', 'equal', '1/2^lambda'], 'local_restart': ['nan', 'IPOP', 'BIPOP'], 'step_size_adaptation': ['csa', 'psr'], 'lambda_': ['5.0', '10.0', '20.0'], 'mu': ['5.0', '10.0', '20.0'], 'active': [False, True]}
[ 5 30]
[0 1 2]
[1 2 3 4 5]
Configuration space object:
  Hyperparameters:
    active, Type: Categorical, Choices: {False, True}, Default: False
    base_sampler, Type: Categorical, Choices: {sobol, gaussian, halton}, Default: sobol
    elitist, Type: Categorical, Choices: {False, True}, Default: False
    lambda_, Type: Categorical, Choices: {5.0, 10.0, 20.0}, Default: 5.0
    local_restart, Type: Categorical, Choices: {nan, IPOP, BIPOP}, Default: nan
    mirrored, Type: Categorical, Choices: {nan, mirrored, mirrored pairwise}, Default: nan
    mu, Type: Categorical, Choices: {5.0, 10.0, 20.0}, Default: 5.0
    step_size_adaptation,

In [2]:
from IPython.display import display

#df = de_explainer.behaviour_stats()
#df.style.bar(cmap='viridis')

#do it for f1 and f5

df = cmaes_explainer.behaviour_stats(per_fid=False)
display(df.style.bar(cmap='viridis'))

#df = cmaes_explainer.behaviour_stats(per_fid=True)
#display(df.style.bar(cmap='viridis'))



In [3]:

cmaes_explainer.to_latex_report(filename="mod_cma_new", img_dir="cma_img_new/")

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
The figure layout has changed to tight
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
The figure layout has changed to tight
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
The figure layout has changed to tight
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
The figure layout has changed to tight
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
The figure layout has ch

<Figure size 800x470 with 0 Axes>

In [21]:
# categorical_columns = cmaes_explainer.df.dtypes[cmaes_explainer.df.dtypes == 'category'].index.to_list()
# categorical_columns

# df = cmaes_explainer.df
# for cat in categorical_columns:
#     print(cat)
#     df[cat] = df[cat].astype('category')
#     print(df[cat].unique())
#     print(df[cat].cat.codes.unique())

mirrored
['mirrored pairwise', NaN, 'mirrored']
Categories (2, object): ['mirrored', 'mirrored pairwise']
[ 1 -1  0]
base_sampler
['sobol', 'gaussian', 'halton']
Categories (3, object): ['gaussian', 'halton', 'sobol']
[2 0 1]
weights_option
['default', 'equal', '1/2^lambda']
Categories (3, object): ['1/2^lambda', 'default', 'equal']
[1 2 0]
local_restart
[NaN, 'IPOP', 'BIPOP']
Categories (2, object): ['BIPOP', 'IPOP']
[-1  1  0]
step_size_adaptation
['csa', 'psr']
Categories (2, object): ['csa', 'psr']
[0 1]


In [47]:
#Correlation of configuration ranking per function
import seaborn as sns
import matplotlib.pyplot as plt


df = cmaes_explainer.df
for dim in cmaes_explainer.dims:
    fid_auc_matrix = []
    for fid in cmaes_explainer.fids:
        df_dim_fid = df[(df['dim']==dim) & (df['fid'] == fid)]
        
        fid_auc_matrix.append(df_dim_fid.groupby(features)['auc'].mean())

    fid_auc_matrix = np.array(fid_auc_matrix).T

    df_matrix = pd.DataFrame(fid_auc_matrix, columns=[f"$f_{{{i}}}$" for i in np.arange(1,25)])
    corr = df_matrix.corr()

    

    g = sns.clustermap(corr, 
                    method = 'complete', 
                    cmap   = 'viridis', 
                    annot  = True, 
                    annot_kws = {'size': 8})
    plt.tight_layout()
    plt.savefig(f"cma-es-clustermap-{dim}d.pdf")
    plt.clf()

    g = sns.heatmap(corr, 
                    annot  = False)
    plt.tight_layout()
    plt.savefig(f"cma-es-heatmap-{dim}d.pdf")
    plt.clf()
#plt.setp(g.ax_heatmap.get_xticklabels(), rotation=60);

The figure layout has changed to tight
The figure layout has changed to tight
The figure layout has changed to tight
The figure layout has changed to tight


<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [24]:
#Correlation of configuration ranking per function
df = cmaes_explainer.df
df_dim_fid = df[(df['dim']==5) & (df['fid'] == 2) & (df['seed'] == 1) & (df['iid'] == 1)]
df_dim_fid

,fid,iid,dim,seed,elitist,mirrored,base_sampler,weights_option,local_restart,step_size_adaptation,auc
4871,2,1,5,1,False,mirrored,gaussian,default,BIPOP,psr,9037.037037
7251,2,1,5,1,False,mirrored,sobol,default,IPOP,csa,9672.839506
9631,2,1,5,1,False,mirrored,halton,equal,BIPOP,csa,9179.012346
12011,2,1,5,1,False,mirrored pairwise,gaussian,default,NaN,psr,9141.975309
16771,2,1,5,1,True,NaN,halton,default,IPOP,psr,9456.790123
...,...,...,...,...,...,...,...,...,...,...,...
753381,2,1,5,1,True,mirrored,sobol,equal,NaN,csa,9703.703704
760521,2,1,5,1,True,mirrored,halton,equal,BIPOP,psr,9697.530864
761711,2,1,5,1,False,mirrored pairwise,gaussian,equal,IPOP,csa,2777.777778
762901,2,1,5,1,False,NaN,gaussian,1/2^lambda,IPOP,psr,9395.061728
